En este código se quiere realizar una implementación de un simulador de físicas sencillo.

In [ ]:
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import sys
import numpy as np
import time as t

#Window size
width = 800
height = 800

xmin=-float(width/2)
xmax=float(width/2)
ymin=-float(height/2)
ymax=float(height/2)

radius = 25.0 #pix

g = np.array([0.0,-0.6])
objects = []

VEL_MAX = 50

class Object:

    def __init__(self, x, y):
        self.velocity = np.array([0,0])
        self.position = np.array([x,y])
        self.vertex = get_circle_points(self,x,y)
        
    def update_physics(self):
        new_v = np.add(self.velocity,g) #updates vel
        if not is_over_limit(new_v): #limit
                self.set_velocity(new_v)
        
        prev_pos = self.position
        new_pos = np.add(self.position,self.velocity) #update pos
        self.set_position(new_pos)
        out_screen(self,new_pos)
        
        #update vertex
        for i in range(len(self.vertex)):
                self.vertex[i] =np.add(self.vertex[i],np.subtract(self.position,prev_pos))
            

    def set_velocity(self,new_vel):
        self.velocity = new_vel
    
    def set_position(self,new_pos):
        self.position = new_pos
    
    def to_string(self):
        return f"{self.position}-{self.velocity}"

    
def get_circle_points(obj,x,y):
    points = []    

    alfa = 0;
    for i in range(360):
        points.append([obj.position[0] + radius * np.cos(alfa), obj.position[1] + radius * np.sin(alfa)])    
        alfa += np.pi / 180.0; # 1 point each degree (in radians)
    return points


def is_over_limit(v):
    return (np.sqrt(np.power(v[0],2) + np.power(v[1],2))) > VEL_MAX
            
# Process the new position on edges
def out_screen(object,new_pos):
    
    if new_pos[0] > (xmax - radius) or new_pos[0] < (xmin + radius):
        object.set_velocity(np.multiply(object.velocity,[-0.75,0.97]))
        if new_pos[0] > (xmax - radius): # DERECHA
            object.set_position(np.add(new_pos,[(xmax - radius)-new_pos[0],0])) 
          
        if new_pos[0] < (xmin + radius): # IZQUIERDA
            object.set_position(np.add(new_pos,[(xmin + radius)-new_pos[0],0]))
        
    if new_pos[1] > (ymax - radius) or new_pos[1] < (ymin + radius):
        object.set_velocity(np.multiply(object.velocity,[0.97,-0.75]))
        
        #if new_pos[1] > (ymax - radius): # ARRIBA
         #  new_pos = np.add(new_pos,[0,(ymax-radius)-new_pos[1]])          
            
        if new_pos[1] < (ymin + radius): # ABAJO
            object.set_position(np.add(new_pos,[0,(ymin+radius)-new_pos[1]]))

            
            
            
            
# OPENGL FUNC

def init():
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glMatrixMode(GL_PROJECTION)
    gluOrtho2D(xmin, xmax, ymin, ymax)
    glMatrixMode(GL_MODELVIEW)


# updates all velocities and positions
def update():
    global objects
    
    t.sleep(0.009) #adds some retard

    for object in objects:
        object.update_physics()
    glutPostRedisplay()
    
    # print("UPDATE")
    # for o in objects:
    # print(o.to_string())
    
    
# on left click creates a ball, on right one deletes all
def mouse(button, state, x, y):
    global objects
    
    realPosMousex = ((xmax-xmin)/width*x+xmin) #convert x,y from window to system x,y
    realPosMousey = ((ymin-ymax)/height*y+ymax)
    
    if button == GLUT_LEFT_BUTTON and state==GLUT_DOWN:
        objects.append(Object(realPosMousex,realPosMousey))

    if button == GLUT_RIGHT_BUTTON and state==GLUT_DOWN:
        objects = []
        
def draw():
    
    #cleans
    glClearColor(0.0, 0.0, 0.0,1.0);
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);
           
        
    #draws all
    for obj in objects:
        glBegin(GL_TRIANGLE_FAN)
        glVertex2fv(obj.position)
        for i in range(len(obj.vertex)):
            glVertex2fv(obj.vertex[i])
        glEnd()
        
    glutSwapBuffers()

    
    
glutInit(sys.argv)

glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB)
# Atributos de la ventana
glutInitWindowPosition(950,200)
glutInitWindowSize(width,height)
glutCreateWindow("Particulas")

init()

glutDisplayFunc(draw)
glutMouseFunc(mouse)
glutIdleFunc(update)
#glutPassiveMotionFunc(cursor)
glutMainLoop()